In [2]:
import pandas as pd
import numpy as np

In [3]:
class normalizeVector():
    def __init__(self, vector, vector2):

        self.vector = vector
        self.refPoint = vector2
        self.refAngle = np.arccos(np.dot(vector, self.refPoint) / (np.linalg.norm(vector) * np.linalg.norm(self.refPoint)))
        self.refAxis = np.cross(vector, self.refPoint) 
        self.refAxis = self.refAxis / np.linalg.norm(self.refAxis)
        self.q = np.sin(self.refAngle / 2) * self.refAxis
        self.q = np.insert(self.q, 0, np.cos(self.refAngle / 2), axis = 0)
        self.p = vector.values
        self.p = np.insert(self.p, 0, 0, axis = 0)
        self.qi = np.array([self.q[0], -self.q[1], -self.q[2], -self.q[3]], dtype=np.float64)

        self.start = self.normalize()


    def quaternion_multiply(self, quaternion1, quaternion0):
        w0, x0, y0, z0 = quaternion0
        w1, x1, y1, z1 = quaternion1
        return np.array([-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                        x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                        -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                        x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0], dtype=np.float64)


    def normalize(self, vector=None):
        if vector is None:
            vector = self.p
        else:
            vector = vector.values
            vector = np.insert(vector, 0, 0, axis = 0)

        result = self.quaternion_multiply(self.quaternion_multiply(self.q, vector), self.qi)[1:] 

        return result

In [4]:
dataSkull = pd.read_csv("../listVskull.csv")
dataSkull.sort_index(axis=1, ascending=True, inplace=True)
dataTree = pd.read_csv("../listVtree.csv")
dataTree.sort_index(axis=1, ascending=True, inplace=True)
dataVent = pd.read_csv("../listVvent.csv")
dataVent.sort_index(axis=1, ascending=True, inplace=True)
dataLow = pd.read_csv("../listVlow.csv")
dataLow.sort_index(axis=1, ascending=True, inplace=True)

In [5]:
rotationMatrix = np.array([[np.cos(np.pi), -np.sin(np.pi), 0],
                           [np.sin(np.pi), np.cos(np.pi), 0],
                           [0,0,1]])
dataSkull = (rotationMatrix @ dataSkull.T).T
dataSkull.columns = ['X', 'Y', 'Z']
dataTree = (rotationMatrix @ dataTree.T).T
dataTree.columns = ['X', 'Y', 'Z']
dataVent = (rotationMatrix @ dataVent.T).T
dataVent.columns = ['X', 'Y', 'Z']
dataLow = (rotationMatrix @ dataLow.T).T
dataLow.columns = ['X', 'Y', 'Z']

In [6]:
mid = rotationMatrix@np.array([-143.5, -12.57, 3.5])
mid

array([143.5 ,  12.57,   3.5 ])

In [7]:
markers = np.array([[143.5, -12.57, 3.5],
                    [145.8, 12, 3.5],
                    [140, -68, 3.5],
                    [185.3, -16.25, 3.5],
                    [144, -12.61, -70]])

In [8]:
dataVec = np.array([420.51,336.515,221.209]) - np.array([391.053,341.058,184.309])
dataVec

array([29.457, -4.543, 36.9  ])

In [9]:
imgVec = markers[3] - markers[0]
imgVec

array([41.8 , -3.68,  0.  ])

In [10]:
normalizer = normalizeVector(pd.Series(imgVec), dataVec)
normalizer.normalize()

array([26.05875145, -4.01890579, 32.64310447])

In [11]:
normalizer.refAxis

array([-0.08757785, -0.99477011, -0.05255989])

In [12]:
s = dataSkull.apply(lambda x: list(normalizer.normalize(x)), axis=1)
dataSkull = pd.DataFrame.from_dict(dict(zip(s.index, s.values))).T
dataSkull.columns = ["X", "Y", "Z"]

In [13]:
s = dataTree.apply(lambda x: list(normalizer.normalize(x)), axis=1)
dataTree = pd.DataFrame.from_dict(dict(zip(s.index, s.values))).T
dataTree.columns = ["X", "Y", "Z"]

In [14]:
s = dataVent.apply(lambda x: list(normalizer.normalize(x)), axis=1)
dataVent = pd.DataFrame.from_dict(dict(zip(s.index, s.values))).T
dataVent.columns = ["X", "Y", "Z"]

In [15]:
s = dataLow.apply(lambda x: list(normalizer.normalize(x)), axis=1)
dataLow = pd.DataFrame.from_dict(dict(zip(s.index, s.values))).T
dataLow.columns = ["X", "Y", "Z"]

In [16]:
mid = normalizer.normalize(pd.Series(mid))
mid

array([ 88.60472616,  11.61549907, 113.03445308])

In [17]:
offset = np.array([391.052, 333.058, 184.3088]) - mid
offset

array([302.44727384, 321.44250093,  71.27434692])

In [18]:
dataSkull = dataSkull + offset
dataTree = dataTree + offset
dataVent = dataVent + offset
dataLow = dataLow + offset

In [172]:
dataSkull.to_csv("../../listVskull.csv", index=False)
dataTree.to_csv("../../listVtree.csv", index=False)
dataVent.to_csv("../../listVvent.csv", index=False)
dataLow.to_csv("../../listVlow.csv", index=False)